# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902072


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:02<00:41,  1.38s/it]

Rendering models:  16%|█▌        | 5/32 [00:05<00:33,  1.22s/it]

Rendering models:  25%|██▌       | 8/32 [00:07<00:24,  1.02s/it]

Rendering models:  34%|███▍      | 11/32 [00:07<00:15,  1.38it/s]

...

Rendering models:  62%|██████▎   | 20/32 [00:07<00:02,  4.55it/s]

Rendering models:  75%|███████▌  | 24/32 [00:07<00:01,  5.98it/s]

Rendering models:  84%|████████▍ | 27/32 [00:07<00:00,  7.83it/s]

Rendering models:  94%|█████████▍| 30/32 [00:08<00:00,  9.61it/s]

equidad1                              0.000262
kayleebug2017                         0.003117
Thedunkmasta                          0.001596
not-logged-in-d254cc3304773762c2aa    0.001395
BirdLeaf                              0.415712
amazedogog                            0.009167
not-logged-in-ab1612bc18cd7aab85db    0.000342
ElisabethB                            0.567293
not-logged-in-2ef7f191c8c973ce093e    0.000211
next_ghost                            0.000208
mattchat                              0.019650
Vbgil13                               0.002321
Paddy_van_CH                          0.000212
not-logged-in-a0164d241f30411408f0    0.000462
GabeGramy                             0.002143
Alexander00                           0.000387
OmGupta                               0.000281
ericmalani                            0.001425
JoseA.G.v                             0.001391
pantatoo                              0.000788
not-logged-in-d9deb066f884063b530c    0.082897
BraxtonKinney

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())